## Notebook to demo function calling with Pydantic response model.

In [1]:

import random

from faker import Faker
from pydantic import BaseModel, Field
from rich.pretty import pprint

from gait import Agent, Dialog, ObserverLoguru

In [2]:
# litellm._turn_on_debug()  # 👈 this is the 1-line change you need to make

In [2]:
class TemperatureModel(BaseModel):
    location: str = Field(
        ...,
        description="The location where to get the temperature."
    )
    temperature: float = Field(
        ...,
        description="The temperature at the location."
    )
    unit: str = Field(
        ...,
        description="The unit of the temperature."
    )

In [3]:
def get_temperature_at_location(location: str) -> TemperatureModel:
    """Get temperature in Fahrenheit at a location.

    :param location: The location where to get the temperature. Location can be a place, city, state, zipcode or a country.
    """
    return TemperatureModel(
        location=location,
        temperature=int(random.uniform(-5, 40) * 10.0) / 10.0,
        unit="F",
    )

In [4]:
fake = Faker()

In [5]:
def logger_fn_pprint(call_dict) -> None:
    pprint(call_dict, expand_all=True)

In [15]:
agent = Agent(
    model="ollama_chat/llama3.2:latest",
    # model="ollama_chat/qwen2.5:7b-instruct-fp16",
    # model=f"azure/{os.environ['AZURE_API_DEPLOYMENT']}",
    # model="ollama_chat/granite3.2:8b-instruct-q8_0",
    functions=[get_temperature_at_location],
    temperature=0.0,
    # logger_fn=logger_fn_pprint,
)

In [24]:
observer = ObserverLoguru()

In [29]:
dialog = Dialog.instance()
dialog += f"Get the temperature in {fake.city()}"

for _ in range(10):
    resp = agent(
        dialog=dialog,
        observer=observer,
    )
    if resp.content:
        print(resp.content)
        break

13:58:23 | INFO | Content: 
13:58:23 | INFO | Function: get_temperature_at_location({"location": "New Patrickview"})
13:58:23 | INFO | Observation: {"location":"New Patrickview","temperature":30.5,"unit":"F"}
/Users/mraad/GWorkspace/gait/.venv/lib/python3.11/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `ChatCompletionMessageToolCall` - serialized value may not be as expected [input_value={'function': {'arguments'...fc', 'type': 'function'}, input_type=dict])
  return self.__pydantic_serializer__.to_python(
13:58:23 | INFO | Content: The current temperature in New Patrickview is 30.5°F (Fahrenheit).


The current temperature in New Patrickview is 30.5°F (Fahrenheit).
